In [1]:
import open_ie_api
from pytorch_pretrained_bert import BertTokenizer, BertForMaskedLM
import torch
import math

ModuleNotFoundError: No module named 'open_ie_api'

In [ ]:
# Load data and models
openbook_data_path = 'data/openbook.txt'
# TODO: load fine-tuned model
bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
def get_score(sentence: str):
    tokenize_input = bert_tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([bert_tokenizer.convert_tokens_to_ids(tokenize_input)])
    predictions = bert_model(tensor_input)
    loss_fct = torch.nn.CrossEntropyLoss()
    loss = loss_fct(predictions.squeeze(), tensor_input.squeeze()).data
    return math.exp(loss)

In [ ]:
# Define methods for aristo and openbook tuples
def print_aristo_scores():
    scores = []
    with open('data/aristo.txt', 'r') as f:
        for line in f:
            sentence = " ".join(line.strip().split('\t'))
            scores.append((sentence, get_score(sentence)))
    scores.sort(key=lambda x: x[1])
    print(scores)

def print_openbookqa_scores():
    scores = []
    with open('outputs/openbook_tuples_old.txt', 'r') as f:
        for line in f:
            sentence = " ".join(line.strip().split('\t')[1:])
            scores.append((sentence, get_score(sentence)))
    scores.sort(key=lambda x: x[1], reverse=True)
    print(*scores, sep='\n')

In [ ]:
# Load openbookqa tuples
tuples = open_ie_api.call_api_many(openbook_data_path)
